In [ ]:
%matplotlib inline
%config InlineBackend.figure_format ='retina'

import torch
import socialforce

# Fit 1+1D

The potential $V(b, d_{\perp})$ is approximated by two 1D potentials:
\begin{align}
    V(b, d_{\perp}) &= \textrm{SF}(b) \cdot \max(0, 1 + a d_{\perp})
\end{align}
where $a$ is the `asymmetry` parameter of the `PedPedPotential2D` constructor.

In [ ]:
V = socialforce.potentials.PedPedPotential2D(asymmetry=-1.0).double()
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

## Circle and ParallelOvertake Scenarios

The preferred speed needs to be varied. Otherwise the symmetry of the problem creates unrealistic scenarios where the two pedestrians get stuck.

In [ ]:
circle = socialforce.scenarios.Circle(ped_ped=V)
parallel = socialforce.scenarios.ParallelOvertake(ped_ped=V)
scenarios = circle.generate(5) + parallel.generate(5)
true_experience = socialforce.Trainer.scenes_to_experience(scenarios)

## MLP

Next we create a model for pedestrian-pedestrian interaction that is the 
product of two 1D potentials: one potential as a function of $b$ and another 
as a function of perpendicular distance. The potential is initialized to random
weights and biases.

\begin{align}
    V(b, d_{\perp}) &= \textrm{MLP}_b(b) \cdot \textrm{MLP}_{\perp}(d_{\perp}) \;\;\; .
\end{align}

In [ ]:
V = socialforce.potentials.PedPedPotentialMLP1p1D().double()

with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)

## Fitting

Next, we use the standard SGD optimizer from PyTorch and train the ped-ped 
interaction model on the synthetic data created above.

In [ ]:
def simulator_factory(initial_state):
    return socialforce.Simulator(initial_state, ped_ped=V)

opt = torch.optim.SGD(V.parameters(), lr=3.0)
socialforce.Trainer(simulator_factory, opt, true_experience).loop(20)

In [ ]:
with socialforce.show.canvas(figsize=(12, 6), ncols=2) as (ax1, ax2):
    socialforce.show.potential2D(V, ax1)
    socialforce.show.potential2D_grad(V, ax2)